In [1]:
import nltk
import numpy as np
import pandas as pd
import re
import seaborn as sns
from bangla_linga import gender_prediction as gp
import os
from google.cloud import translate
from google.cloud import storage
from nltk.tokenize import sent_tokenize, word_tokenize
import editdistance

In [2]:
from sklearn.linear_model import LogisticRegression
import pickle
model = pickle.load(open("model.sav", 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: 'model.sav'

In [43]:
df = pd.read_csv('file_big.csv')
df.drop([df.columns[0],'em1','em2','em3','href','file','date','weight'],axis=1,inplace=True)
df.drop_duplicates(inplace= True)
df.dropna(inplace=True)
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')

<AxesSubplot:>

In [ ]:
df['sex'] = ""
df['sex'] = np.nan
df

In [ ]:
gen = gp.BN_gen_pred()
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"thesis-translate-57c203628831.json"
def translate_text(target, text):
    import six
    from google.cloud import translate_v2 as translate

    translate_client = translate.Client()

    if isinstance(text, six.binary_type):
        text = text.decode("utf-8")
    result = translate_client.translate(text, target_language=target)
    return result["translatedText"]

In [ ]:
for j, k in df.iterrows():
    commenter = str(df.name[j]).partition(' ')[0].lower()
    df.sex[j] = gen.predict_gender(translate_text('bn',commenter))
    
df

In [ ]:
df.to_csv('dataset.csv', index = False)

In [ ]:
products = pd.read_csv('phone_list.csv')

In [20]:
df_labeled = pd.read_csv('dataset.csv')
df_labeled.dropna(axis=0, inplace = True)
df_labeled['bo'],df_labeled['eo'],df_labeled['sentiment'],df_labeled['entity'] = "","","",""
df_labeled['bo'],df_labeled['eo'],df_labeled['sentiment'],df_labeled['entity'] = np.nan,np.nan,np.nan,np.nan
df_labeled

,name,coment,sex,bo,eo,sentiment,entity
0,Abirul Islam,iphone 10 pro 64MP/108MP??,male,NaN,NaN,NaN,NaN
1,Rana Ahmed,Poco X3 pro official ki international??,male,NaN,NaN,NaN,NaN
2,Murad Khan,"redmi 10 pro max, sathe kono gift ache?",male,NaN,NaN,NaN,NaN
3,Md Faysal Ahmed,Poco x3 Indian version ta ki ase... ??,female,NaN,NaN,NaN,NaN
4,Md Rashedujjaman Reza,redmi Note 10 pro 8/128 gb available nai?,female,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
3000,Raihan Raj,Poco x3 pro 8/256 unofficial ache?,male,NaN,NaN,NaN,NaN
3001,শুভ্র শুভ্র,Pixel 4a 5g which variants,female,NaN,NaN,NaN,NaN
3002,Sheikh Shakib,??,male,NaN,NaN,NaN,NaN
3003,Mahmudul Hassan Shuvro,Redmi k30 ultra available?,female,NaN,NaN,NaN,NaN


In [ ]:
df_labeled.to_csv('dataset-labeled.csv')

In [17]:
df_labeled = pd.read_csv('dataset-labeled.csv')

In [18]:
df_labeled

,Unnamed: 0,model,comment,sentiment,sex
0,0,Realme 8,realme 8 die selfie den keu,1,0.0
1,1,Pixel 4,Pixel 4 Supports dual SIM cards from different...,1,0.0
2,2,iPhone 12,face id is not perfect in iphone 12,0,0.0
3,3,Poco x3,keu amake ektu poco x3 er features gula bolen,1,1.0
4,4,Mi Note 10,redmi note 10 6/128 find palm komer sombabona ...,1,1.0
...,...,...,...,...,...
5219,5219,Realme 8,realme 8 use kortesi dual band wifi valoi netw...,1,1.0
5220,5220,Redmi 9,validity chek krsen redmi 9 er,1,1.0
5221,5221,Realme 8,premium look purai realme 8 er,1,1.0
5222,5222,Realme 8,"realme 8 fast charge hoy, 30 min e full charge",1,0.0


In [ ]:
#Analyzing sentiments of dataset

In [96]:
df_sentiment = pd.read_csv('labeled-set.csv')
df_sentiment.head()

,Unnamed: 0,model,comment,sentiment,sex
0,0,iPhone 10,iphone 10 oppo 64MP/108MP ? ?,1,0
1,1,POCO X3,poco X3 oppo official ki international ? ?,1,0
2,2,Redmi 1,"redmi 10 oppo mate , mate honor gift ache ?",1,1
3,3,POCO X3,poco x3 Indian version ta ki mate ... ? ?,1,0
4,4,redmi note 10,redmi note 10 oppo 8/128 gb available nai ?,1,1


In [97]:
review_df = df_sentiment[['comment','sentiment']]
review_df = review_df.iloc[:4650,:]
print(review_df.shape)
review_df.head(5)

(4650, 2)


,comment,sentiment
0,iphone 10 oppo 64MP/108MP ? ?,1
1,poco X3 oppo official ki international ? ?,1
2,"redmi 10 oppo mate , mate honor gift ache ?",1
3,poco x3 Indian version ta ki mate ... ? ?,1
4,redmi note 10 oppo 8/128 gb available nai ?,1


In [104]:
review_df["sentiment"].value_counts()

1    4633
0      17
Name: sentiment, dtype: int64

In [103]:
sentiment_label = review_df.sentiment.factorize()
print(sentiment_label[0][:10])

[0 0 0 0 0 0 0 0 0 0]


In [105]:
comments = review_df.comment.values

In [106]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(comments)

In [107]:
encoded_docs = tokenizer.texts_to_sequences(comments)

In [108]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_sequence = pad_sequences(encoded_docs, maxlen=300)

In [109]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding
embedding_vector_length = 128
model = Sequential()
model.add(Embedding(10000, embedding_vector_length, input_length=300))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 300, 128)          1280000   
_________________________________________________________________
spatial_dropout1d_7 (Spatial (None, 300, 128)          0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 50)                35800     
_________________________________________________________________
dropout_7 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 51        
Total params: 1,315,851
Trainable params: 1,315,851
Non-trainable params: 0
_________________________________________________________________
None


In [110]:
history = model.fit(padded_sequence,sentiment_label[0],validation_split=0.2, epochs=4, batch_size=32)

Train on 3720 samples, validate on 930 samples
Epoch 1/4
3720/3720 [==============================] - 87s 23ms/sample - loss: 0.1009 - accuracy: 0.9884 - val_loss: 0.1347 - val_accuracy: 0.9817
Epoch 2/4
3720/3720 [==============================] - 96s 26ms/sample - loss: 7.5561e-04 - accuracy: 1.0000 - val_loss: 0.1494 - val_accuracy: 0.9817
Epoch 3/4
3720/3720 [==============================] - 85s 23ms/sample - loss: 3.6895e-04 - accuracy: 1.0000 - val_loss: 0.1579 - val_accuracy: 0.9817
Epoch 4/4
3720/3720 [==============================] - 86s 23ms/sample - loss: 2.4045e-04 - accuracy: 1.0000 - val_loss: 0.1641 - val_accuracy: 0.9817


In [122]:
def predict_sentiment(text):
    tw = tokenizer.texts_to_sequences([text])
    tw = pad_sequences(tw,maxlen=300)
    prediction = model.predict(tw).item()
    if prediction < 0.000121:
        return 0
    else:
        return 1

In [123]:
df_test = pd.read_csv('test.csv')
accuracy = 0
match = 0
count = 0
for j, k in df_test.iterrows():
    sentiment = int(df_test.sentiment[j])
    test_text = str(df_test.comment[j])
    

    sen = int(predict_sentiment(test_text))

    if (sen == sentiment):
        match = match + 1
    count = count + 1
accuracy = match/count*100
print(accuracy)

86.02870813397129


In [95]:
df_test = pd.read_csv('new_trainset6.csv')
df_test.drop([df_test.columns[0]],axis=1,inplace=True)
df_test.to_csv('labeled-set.csv')

In [172]:
df_labeled.to_csv('dataset-labeled.csv')

In [ ]:
#Removing non product mentioned comments

In [149]:
df_labeled = pd.read_csv('dataset-labeled.csv')

In [174]:
for j,k in df_labeled.iterrows():
    comment = df_labeled.coment[j].lower()
    regexmatch = re.search('^((?!iphone)xs|xr|(1[1-3]\s*(pro|max|mini|pro max|[0-9]+GB)))$',comment,re.IGNORECASE)
    if regexmatch:
        print(comment)
        refined_com = re.sub('((?!iphone)xs|xr|(1[1-3]\s*(pro|max|mini|pro max|[0-9]+GB)))', 'iphone '+regexmatch.group(0), comment,re.IGNORECASE)
        print(refined_com)
        #df_labeled.coment[j] = refined_com

apple watch series3 price koto hobe
apple watch series 3 price koto hobe


/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [138]:
for j,k in df_labeled.iterrows():
    comment = df_labeled.coment[j].lower()
    regexmatch = re.search('\s*\+',comment,re.IGNORECASE)
    if regexmatch:
        print(comment)
        refined_com = re.sub('\s\+', ' plus', comment,re.IGNORECASE)
        print(refined_com)
        #df_labeled.coment[j] = refined_com

In [145]:
for j,k in df_labeled.iterrows():
    comment = df_labeled.coment[j].lower()
    regexmatch = re.search('note\s[0-9][0-9]max',comment,re.IGNORECASE)
    if regexmatch:
        print(comment)
        refined_com = re.sub('note\s[0-9][0-9]max', 'note 10 max', comment,re.IGNORECASE)
        print(refined_com)
        #df_labeled.coment[j] = refined_com    

eto daam keno? note 10max to onno ra 26k te dicche?
eto daam keno? note 10 max to onno ra 26k te dicche?


/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
import spacy

# The source pipeline with different components
source_nlp = spacy.load("en_core_web_sm")
print(source_nlp.pipe_names)

# Add only the entity recognizer to the new blank pipeline
nlp = spacy.blank("en")
nlp.add_pipe("ner", source=source_nlp)

In [ ]:
nlp.begin_training()

In [ ]:
ner=nlp.get_pipe("ner")

In [ ]:
LABEL = "model"
TRAIN_DATA = [
              ("Huwaei honor 10 lite new or used? & available in which branch?", {"entities": [(0, 14, "model") ]}),
              ("HUAWEI mate 40 pro  Is it possible!  If possible, price please.", {"entities": [(0, 17, "model") ]}),
              ("When ZTE Nubia Red magic 6 pro will be available?", {"entities": [(5,29, "model") ]}),
              ("OnePlus 7 Pro Price ???", {"entities": [(0,12, "model") ]}),
              ("oneplus 7t hobe vai?", {"entities": [(0,9, "model") ]}),
              ("Vai brand new Oneplus 8 pro (12/256) patuakhali te shunderban curiyar shoho koto tk porbe...", {"entities": [(14,26, "model") ]}),
              ("oneplus nord price?", {"entities": [(0,11, "model") ]}),
              ("oppo A92 sell kora jabe?", {"entities": [(0,7, "model") ]}),
              ("In Google Pixel 4A, Which variant you are selling?", {"entities": [(3,17, "model") ]}),
              ("pixel 4a 5g white hobe?", {"entities": [(0,10, "model") ]}),
              ("kew ikto janaben  realme c17 ar price bortoman a koto?? janale onek bhalo hoto", {"entities": [(18,27, "model") ]}),
              ("Realme C25 available?", {"entities": [(0,9, "model") ]}),
              ("Realme x3 superzoom?", {"entities": [(0,18, "model") ]}),
              ("Realme 5i à¦à¦° à¦¡à¦¿à¦¸à¦ªà§à¦²à§‡ à¦­à§‡à¦™à§à¦—à§‡ à¦—à§‡à¦›à§‡ , à¦†à¦ªà¦¨à¦¾à¦¦à§‡à¦° à¦•à¦¾à¦›à§‡ à¦•à¦¿ à¦à¦° à¦¡à¦¿à¦¸à¦ªà§à¦²à§‡ à¦ªà¦¾à¦“à¦¯à¦¼à¦¾ à¦¯à¦¾à¦¬à§‡ ?", {"entities": [(0,8, "model") ]}),
              ("Realme 5i à¦à¦° à¦¡à¦¿à¦¸à¦ªà§à¦²à§‡ à¦­à§‡à¦™à§à¦—à§‡ à¦—à§‡à¦›à§‡ , à¦†à¦ªà¦¨à¦¾à¦¦à§‡à¦° à¦•à¦¾à¦›à§‡ à¦•à¦¿ à¦à¦° à¦¡à¦¿à¦¸à¦ªà§à¦²à§‡ à¦ªà¦¾à¦“à¦¯à¦¼à¦¾ à¦¯à¦¾à¦¬à§‡ ?", {"entities": [(0,8, "model") ]}),
              ("realme 6 hbe???", {"entities": [(0,7, "model") ]}),
              ("Realme x3 superzoom?", {"entities": [(0,18, "model") ]}),
              ("Realme Narzo 20pro used phone er price koto?",{"entities": [(0,17, "model") ]}),
              ("Realme x3 superzoom?", {"entities": [(0,18, "model") ]}),
              ("kew ikto janaben  realme c17 ar price bortoman a koto?? janale onek bhalo hoto", {"entities": [(18,27, "model") ]}),
              ("redmi 10x chinese version e kon processor?", {"entities": [(0,8, "model") ]}),
              ("à¦­à¦¾à¦‡ redmi 9 unofficial à¦¹à¦¬à§‡? à¦¹à¦²à§‡ à¦¦à¦¾à¦®à¦Ÿà¦¾ à¦œà¦¾à¦¨à¦¾à¦¬à§‡à¦¨ à¦ªà§à¦²à¦¿à¦œà¥¤", {"entities": [(10,16, "model") ]}),
              ("Redmi k30 ultra available?", {"entities": [(0,14, "model") ]}),
              ("Samsung galaxy S20 fe 4g snapdragon varrient ta ki available hbe? And hole price?", {"entities": [(0,17, "model") ]}),
              ("I want to sell Samsung Galaxy Note10+, Exynos,- 12GB+256GB, purchase feb,19,2021. Bsundhara city. GM Petrobangla, 01711762932 everything provided cash memo imei matching box cover etc", {"entities": [(15,36, "model") ]}),
              ("Samsung Galaxy A52 à¦à¦° à¦¦à¦¾à¦® à¦¦à¦¿à§Ÿà§‡ à¦°à§‡à¦–à§‡à¦›à§‡à¦¨, à¦•à¦¿à¦¨à§à¦¤à§ à¦†à¦ªà¦¨à¦¾à¦¦à§‡à¦° à¦¶à¦ª à¦ à¦«à§‹à¦¨ à¦¦à¦¿à¦²à¦¾à¦® à¦¬à§‹à¦²à§à¦²à§‡à¦¨ à¦¨à¦¾à¦‡!!", {"entities": [(0,17, "model") ]}),
              ("Samsung galaxy S20 fe 4g snapdragon varrient ta ki available hbe? And hole price?", {"entities": [(0,17, "model") ]}),
              ("Samsung Galaxy M02s Price Koto", {"entities": [(0,18, "model") ]}),
              ("Vivo v20 white available???  Price please", {"entities": [(0,7, "model") ]}),
              ("Poco m2 pro (6/64)ki ase prise koto", {"entities": [(0,10, "model") ]}),
              ("Poco x3 nfc 6/128 available?", {"entities": [(0,11, "model") ]}),
              ("Poco m2 pro (6/64)ki ase prise koto", {"entities": [(0,10, "model") ]}),
              ("POCO X3 Indian available??", {"entities": [(0,6, "model") ]}),
              ("Poco M3 6/128Gb variant  ki anben?", {"entities": [(0,6, "model") ]}),
              ("Poco x3 pro 6/128 international global price Please", {"entities": [(0,10, "model") ]}),
              ("vai poco f3 ase??", {"entities": [(4,10, "model") ]}),
              ("Poco m2 reloaded phone er price", {"entities": [(0,15, "model") ]}),
              ("à¦†à¦šà§à¦›à¦¾ à¦­à¦¾à¦‡à§Ÿà¦¾ Xiaomi Redmi not 10 pro@ iPhone 7+ à¦•à§‹à¦¨à¦Ÿà¦¾ à¦­à¦¾à¦²à§‹ à¦¹à¦¬à§‡", {"entities": [(32,54, "model") ]}),
              ("iPhone 12 mini which country? iPhone SE2 available?", {"entities": [(0,13, "model") ]}),
              ("iPhone 12 Pro Max 256GB Dual (Active) ki available?", {"entities": [(0,12, "model") ]}),
              ("iPhone 12 mini which country? iPhone SE2 available?", {"entities": [(0,13, "model") ]}),
              ("iPhone 12 mini which country? iPhone SE2 available?", {"entities": [(0,13, "model") ]}),
              ("Why we pick you when others giving iphone 11 usa on 64,500 taka with 100% authentic and inactive?", {"entities": [(35,43, "model") ]}),
              ("Iphone xs er price kmn?", {"entities": [(0,8, "model") ]}),
              ("iPhone xr kto?", {"entities": [(0,8, "model") ]}),
              ("Iphone xs er price kmn?", {"entities": [(0,8, "model") ]}),
              ("Note 10 pro 64 MP edition international global ase ki",{"entities": [(0,10, "model") ]})
              ]

In [ ]:
TRAIN_DATA[5]

In [ ]:
ner.add_label(LABEL)
optimizer = nlp.resume_training()
move_names = list(ner.move_names)

In [ ]:
import random
from spacy.util import minibatch, compounding
from pathlib import Path

In [ ]:
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [ ]:
from spacy.util import minibatch, compounding
import random

# Begin training by disabling other pipeline components
with nlp.disable_pipes(*other_pipes) :

  sizes = compounding(1.0, 4.0, 1.001)
  # Training for 30 iterations     
  for itn in range(10):
    # shuffle examples before training
    random.shuffle(TRAIN_DATA)
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=sizes)
    # ictionary to store losses
    losses = {}
    for batch in batches:
      texts, annotations = zip(*batch)
      # Calling update() over the iteration
      nlp.update(batch)
      print("Losses", losses)

In [ ]:

examples = []
for text, annots in TRAIN_DATA:
    examples.append(example.from_dict(nlp.make_doc(text), annots))
nlp.initialize(lambda: examples)
for i in range(20):
    random.shuffle(examples)
    for batch in minibatch(examples, size=8):
        nlp.update(batch)

In [20]:
txt = "The rain in Spain"
x = re.search("Spain", txt)
print(x)

<re.Match object; span=(12, 17), match='Spain'>
